# Creating command line tools CLI

## The `click` module

For System Administrators, command line tools are their daily bread. This section helps to create beautiful command line interfaces in a composable way with as little code as necessary. The [click module](https://click.palletsprojects.com/en/7.x/) is the most popular among the many that exist, e.g. [argparse](https://click.palletsprojects.com/en/7.x/why/#why-not-argparse), [docopt](https://click.palletsprojects.com/en/7.x/why/#why-not-docopt-etc) among others.

The click module not only makes it relatively easy to create nice and extendible command line interfaces, but also implements many best practices. For example, it automatically creates a help page for you (for every command and subcommand), handles flags and passwords in a consistent and predictable way and generates human-friendly errors.

But before we start, we need to understand how command line utilities usually work. They consists of three parts:

* `command`
* `option(s)`
* `argument`

The `command` is the verb, it explains what we would like to do, e.g. **import-datasets** or **copy-logfiles**. Important: it is not necessary to have a command when your utility is just doing one thing.

The `options` (one, many or none) define more precisely how a utility should behave. Options start with either:
* one dash followed by a letter, e.g. `-a`, `-v`
* two dashes followed by a word,  e.g. `--add`, `--verbose`
Typically most options have a short and a long version, so users can either write `-a` or `--add`. Options can be followed by an argument or can act as a switch, like the `--verbose` switch

The `argument` is the subject, often a filename, but it can be a simple string too. The `argument` follows after each `option`, but can also follow after a command.

### Installation of `click`

The installation of the module, using the `pip` utility, is easy. Note that
* `install` is the command
* `-U` is an option, a shorthand for `--upgrade` which forces pip to install the latest version of that module
* `click` is the argument, in our case the name of the module

Documentation of the module can be found [here](https://click.palletsprojects.com/en/7.x/).

In [ ]:
!pip install -U click --user

### Your first CLI tool

Your first module does a lot of useful things:

* it prompts for your name
* it greets you the many times you want
* it offers a polite version

Open the file [hello_click.py](hello_click.py) or look at the code below.

```python
import click

@click.command()
@click.option('--count',  '-c', default=1, help='Number of greetings.')
@click.option('--polite', '-p', is_flag=True)
@click.option('--name',   '-n', prompt='Your name', help='The person to greet.')
def hello(count, polite, name):
    """Simple program that greets NAME for a total of COUNT times."""

    if polite:
        greeting = 'Your Serene Highness'
    else:
        greeting = 'Hello'
        
    for x in range(count):
        click.echo(f'{greeting} {name}!')

if __name__ == '__main__':
    hello()
```

### Understanding the decorators

The first line we already understand:

```python
import click
```

This just imports the click module, ready to be used.

Next, we discover some new elements:

```python
@click.command()
@click.option('--count',  '-c', default=1, help='Number of greetings.')
@click.option('--polite', '-p', is_flag=True)
@click.option('--name',   '-n', prompt='Your name', help='The person to greet.')
def hello(count, polite, name):
```

These elements are so called **decorators**. Their functionality is to **wrap** a function or a method. Before the `def hello()` function is being executed, first the `click.command()` is executed, then three times a `click.option()` decorator, before we actually see the function definition.

The options `--count`, `--polite` and `--name` are passed as parameters `count`, `polite` and `name` to our `hello` function.

You can also define kebab-style options, e.g. `--test-only` which then will be passed as a `test_only` parameter to your function.

### The entry point of your script: `__name__ == '__main__'`

One of the more confusing things in Python is this pattern:

```python
if __name__ == '__main__':
    hello()
```

Try to remove this code from your [hello_click.py](hello_click.py) script and then run it with Python interpreter `python hello_click.py`. What happens?

**Nothing**. Really?

The Python interpreter indeed runs your script. After the syntax check, it first imports the click module, then it wraps our `hello()` function with 4 decorators, makes the function `hello` available to the program. Finally it reaches the end of the script and exits.

We have to tell the Python interpreter what to do next, e.g. execute our function. For this we could simply add

```python
hello()
```

at the end of our script, that function would **always be executed**, especially when we import the script from another script. Open [import_hello_click.py](import_hello_click.py) and then run it: `python import_hello_click.py`.

**Explanation**

When importing a script (or a module), all code gets compiled. If there is any function call, that function gets executed too. The special variable `__name__` contains the value `__main__` if the script was the first thing that got started. It contains the module name `hello_click` if the script got imported.

### Excercise

modify hello_click.py

- add a help text for the `--polite` option and test it
- add a file **argument** which also tests the existence of the file
- use the [documentation](https://click.palletsprojects.com/en/7.x/arguments/#file-path-arguments) how to write that decorator

## Obtaining options

[Options](https://click.palletsprojects.com/en/8.0.x/options/) are - as the name suggests - optional. In addtion options are not positional, they can be provided in any order. Your CLI should always work without any options given. If your CLI needs at least one value, use [Arguments](https://click.palletsprojects.com/en/8.0.x/arguments/) instead (see below)

### String values

As we learned already, we can obtain options from the command line via `@click.option` decorators, e.g.

```python
@click.option("--name", prompt="Your name", help="The person to greet.")
```

If we want to use this method to generate a prompt and directly assign an input value to a variable, we would use `click.prompt()` instead:

In [ ]:
import click

name = click.prompt(text="Your name")

print(f"Hello {name}!")

### Boolean values (flags)

Often we want to adjust the behaviour of our command line tool by providing boolean flags. The presence of the flag will set it to True:

```python

@click.option(
    '-d', '--delete',
    is_flag=True,
    help='delete this user'
)
```



### Passwords

When you enter a password or some other confidential value, you don't want to see on the screen what you write. The `hide_input=True` parameter will prevent this.

When resetting passwords, one often needs to enter the new password twice, in case you entered a typo. Use `confirmation_prompt=True` to show the password prompt a second time and compare the two inputs.

```python
@click.option(
    '--password', 
    prompt=True,                 # does what it says: it prompts, it actively waits for input
    confirmation_prompt=True,    # the password has to be entered twice
    hide_input=True              # do not show the input characters, of course...
)
```


### Exercise 2

Open again [hello_click.py](hello_click.py)

- add a password option to obtain a password
- if no name is provided, instead of prompting, use the username
- hint 1: use the `default` parameter
- hint 2: use the `lambda: ` and the `os.environ` module to obtain the 'USER' environment variable.

## Arguments

[Arguments](https://click.palletsprojects.com/en/8.0.x/arguments/) work almost like options with the difference that they are *positional*. By adding the parameter `nargs=-1`, you can add as many values for one argument (only works for *one* argument):

Example:

```python
@click.command()
@click.argument('src', nargs=-1)
@click.argument('dst', nargs=1)
def copy(src, dst):
    """Move file SRC to DST."""
    for fn in src:
        click.echo(f"move {fn} to folder {dst}")
```

## Commands

Commands are a way to conveniently group options for a given task. For example, the [ethz-iam-webservice](https://gitlab.ethz.ch/vermeul/ethz-iam-webservice) CLI offers the `person`, `user` and the `group` command. Below you see some examples how the utility takes a **command** followed by an **argument** and some **options**:

```bash
$ iam person swen@ethz.ch
$ iam user vermeul
$ iam user vermeul --grant-service LDAPS
$ iam group my_group
$ iam group my_group -a new_user1 -a new_user2 -r old_user3
$ iam group my_group --delete
```

To **implement a command**:

1. define the main entry function, e.g. `def do_something:` and decorate it with the `@click.group()` decorator
2. define the subcommand function, e.g. `def sing:` and decorate it with the `@do_something.command()` (not `@click.command()`!) decorator

### Exercise 3

Open [click_commands.py](click_commands.py)

- observe how the two sub commands are defined
- try to pass some options and arguments to the commands
- implement an example which takes a value of the main option and passes it to the command

# register a command line utility

When you install the [ethz-iam-webservice]((https://gitlab.ethz.ch/vermeul/ethz-iam-webservice/-/tree/master)) utility using `pip install ethz-iam-webservice --user`, you will end up with a command line tool named `iam`. How did that happen?

When you run a `pip install`, the included `setup.py` script is executed. It contains a lot of options, among them the [entry_points](https://setuptools.pypa.io/en/latest/userguide/entry_point.html) with the `console_scripts`:

```python
    entry_points={
        'console_scripts' : [
            'iam=ethz_iam_webservice.main:cli'
        ]
    }
```

The setuptools will create a small python script (a wrapper script) for you and store it in the `bin/` directory next to your Python interpreter. That Python script has the name `iam` which executes `ethz_iam_webservice.main:cli`:

```
ethz_iam_webservice.main:cli
      ^              ^    ^
      |              |    |
  module name   main.py  method name
```

Instead of including it into `setup.py`, you could also create a configuration file `setup.cfg`:

```
[options.entry_points]
console_scripts =
    iam = ethz_iam_webservice.main:cli
```

### Exercise 4

1. create a folder `my_doings` and move or copy `click_commands.py` into that folder
2. inside that folder, create an empty file `__init__.py` to indicate `my_doings` is actually a Python module
3. in the `ws3/` folder, create a configuration file `setup.cfg` with the following content:

```
[metadata]
name = my_doings
version = 0.0.1

[options]
packages = my_doings
```

4. Add the [options.entry_points] section to create a globally available command-line tool `do_something` (see example above)

5. in the `ws3/` folder, create a project file `pyproject.toml` with the following content:

```
[build-system]
requires = ["setuptools", "wheel"]
build-backend = "setuptools.build_meta"
```

6. install the utility with `pip install --editable . --user`
7. try it out! The command `do_something` should now be available inside the console

# working with the shell from Python

While `click` module allows us to create complex command line programs, we still often have to rely on existing tools, i.e. **we need to execute a command on the shell** from within Python. To achieve this, we use the [subprocess module](https://docs.python.org/3/library/subprocess.html), a module from the standard library which faced a lot of changes in the past and is sometimes not very straight forward to use.

## Execute a command and capture the output

the following **only works with Python 3.7 and onwards:** 

In [ ]:
import subprocess
result = subprocess.run(["ls", "-la"], capture_output=True)
print(result.stdout.decode("utf-8"))   # stdout is in bytes, we need to decode it to utf-8

For **up to Python 3.6** use this:

In [ ]:
import subprocess
result = subprocess.run(["ls", "-la"], stdout=subprocess.PIPE)
print(result.stdout.decode("utf-8"))   # stdout is in bytes, we need to decode it to utf-8

## Pipe a command to another 

In Unix, piping the results of one command into another is one of the biggest inventions since the invention of the surfboard. Unfortunately, in Python, piping is not very straight-forward to get it right:

1. start a subprocess p1 for the first command
2. retrieve the `stdout` from p1
3. start a subprocess p2, feed the `p1.stdout` into the `stdin` of p2
4. close the `stdout` of p1
5. invoke the `p2.communicate()` method
6. retrieve the output of p2

Here is a simple example which actually works, so you don't have to google it yourself.

In [ ]:
# Imitate piping on the shell
import subprocess
p1 = subprocess.Popen(["ls", "-la"], stdout=subprocess.PIPE)
p2 = subprocess.Popen(["grep", "hello"], stdin=p1.stdout, stdout=subprocess.PIPE)
p1.stdout.close()  # Allow p1 to receive a SIGPIPE if p2 exits.
output = p2.communicate()[0]
print(output.decode('utf-8'))